<a href="https://colab.research.google.com/github/jraval/Unit2BuildProjectLambda/blob/main/Unit2BuildDSPT10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
import os
print(os.getcwd())

/content


In [21]:
print(os.listdir(os.getcwd()))

['.config', 'KaggleFallDetection_2_8_21.csv', 'sample_data']


In [22]:
from google.colab import files
uploaded = files.upload()

In [25]:
data = pd.read_csv("KaggleFallDetection_2_8_21.csv") 

In [26]:
data

,ACTIVITY,TIME,SL,EEG,BP,HR,CIRCLUATION
0,3,4722.92,4019.64,-1600.00,13,79,317
1,2,4059.12,2191.03,-1146.08,20,54,165
2,2,4773.56,2787.99,-1263.38,46,67,224
3,4,8271.27,9545.98,-2848.93,26,138,554
4,4,7102.16,14148.80,-2381.15,85,120,809
...,...,...,...,...,...,...,...
16377,4,9280.68,11417.00,-3021.64,36,156,654
16378,3,8479.69,9455.54,-2932.85,17,138,554
16379,2,8872.53,27449.90,-2870.00,33,156,1364
16380,4,7738.99,26466.40,-2920.24,97,156,1521
